<a href="https://colab.research.google.com/github/jlyng091/cross_border_financial_flows/blob/master/bertrand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

import numpy as np
import os
import time

TensorFlow 2.x selected.
2.0.0


In [2]:
# Import data
from google.colab import files
pop = files.upload()
aom = files.upload()
koem = files.upload()
mal = files.upload()
howf = files.upload()
aoma = files.upload()

Saving problems_of_philosophy.txt to problems_of_philosophy.txt


Saving analysis_of_mind.txt to analysis_of_mind.txt


Saving knowledge_of_external_world.txt to knowledge_of_external_world.txt


Saving mysticism_and_logic.txt to mysticism_and_logic.txt


Saving history_of_western_philosophy.txt to history_of_western_philosophy.txt


Saving analysis_of_matter.txt to analysis_of_matter.txt


In [3]:
upload = list(pop.values()) + list(aom.values()) + list(koem.values()) + list(mal.values()) + list(howf.values()) + list(aoma.values())
corpus = [book.decode('utf-8') for book in upload]
corpus = corpus[0] + corpus[1] + corpus[2] + corpus[3] + corpus[4] + corpus[5]
len(corpus)

3135682

In [4]:
unique_chars = sorted(set(corpus))  # should we lowercase? the network should pick up on period-uppercase
len(unique_chars)

144

In [5]:
# Character Representation
char2idx = {u:i for i, u in enumerate(unique_chars)}
idx2char = np.array(unique_chars)

text_as_int = np.array([char2idx[char] for char in corpus])
print ('{} ---- characters mapped to int ---- > {}'.format(repr(corpus[:13]), text_as_int[:13]))

'PROBLEMS OF P' ---- characters mapped to int ---- > [46 48 45 32 42 35 43 49  1 45 36  1 46]


In [0]:
# Training examples and targets

# define window
window = 99
sequences_per_epoch = len(corpus)//(window + 1)

# from_tensor_slices creates a tensorflow dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# dataset method to batch our chars into windows of 100, dropping the remainder
sequences = char_dataset.batch(window + 1, drop_remainder = True)

def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]         # implicitly defining a stride of 1

  return input_text, target_text

dataset = sequences.map(split_input_target)

In [7]:
# Shuffle data for SGD

batch_size = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(batch_size, drop_remainder = True)
dataset

<BatchDataset shapes: ((64, 99), (64, 99)), types: (tf.int64, tf.int64)>

In [8]:
# Building the model
vocab_size = len(unique_chars)
embedding_dim = 256
rnn_units = 1024

# params = {dropout:[0.0, 0.1, 0.2, 0.3, 0.4 0.5]}

# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#   model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                               batch_input_shape=[batch_size, None]),
#     tf.keras.layers.LSTM(rnn_units,
#                         return_sequences=True,
#                         stateful=True,
#                         recurrent_initializer='glorot_uniform'),
#     tf.keras.layers.Dense(vocab_size)
#   ])
#   return model

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
                        # dropout=0.3),
                        # recurrent_dropout=0.1),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
                        # dropout=0.3),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
                        # dropout=0.3),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
                        # dropout=0.3),
    tf.keras.layers.Dense(vocab_size),
  ])
  return model

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           36864     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
gru_3 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (64, None, 144)           147600    
Total params: 23,015,568
Trainable params: 23,015,568
Non-trainable params: 0
____________________________________________

In [0]:
# Compile model

def loss(labels, logits):
  return tf.keras.losses.     (labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [10]:
# Train model
EPOCHS = 60
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
489/489 [==============================] - 100s 204ms/step - loss: 1.7997
Epoch 2/60
489/489 [==============================] - 92s 189ms/step - loss: 1.1725
Epoch 3/60
489/489 [==============================] - 92s 189ms/step - loss: 1.0877
Epoch 4/60
489/489 [==============================] - 92s 189ms/step - loss: 1.0355
Epoch 5/60
489/489 [==============================] - 93s 189ms/step - loss: 0.9962
Epoch 6/60
489/489 [==============================] - 93s 189ms/step - loss: 0.9637
Epoch 7/60
489/489 [==============================] - 92s 189ms/step - loss: 0.9361
Epoch 8/60
489/489 [==============================] - 93s 189ms/step - loss: 0.9119
Epoch 9/60
489/489 [==============================] - 93s 189ms/step - loss: 0.8902
Epoch 10/60
489/489 [==============================] - 92s 189ms/step - loss: 0.8729
Epoch 11/60
489/489 [==============================] - 92s 189ms/step - loss: 0.8578
Epoch 12/60
489/489 [==============================] - 93s 189ms/step - l

KeyboardInterrupt: ignored

In [1]:
# Rebuild model for text-generation
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

NameError: ignored

In [17]:
# Text Generation

def generate_text(model, start_string):
  num_generate = 3000

  # start_string conversion
  input_eval = [char2idx[char] for char in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  # low = predictable, high = surprising
  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    # sample text from the distribution
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # pass prediction and hidden state to next input
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string='There are those who take mental \
phenomena naively, just as they would physical phenomena. This school \
of psychologists tends not to emphasise the object.'))

There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasise the object.
Between any two points, these others are irreducible. 
But from an atomic, or such as knowing (i, and
they be formerly and structure, and nothing change of mass. Throughout 
this, 

The notive is so many-one, result for only 
sensible. So long as the sequence is noted. They s It would seem, they will sure-the hypothesis affords each other in any system of co-ordinates 
and by kinds in any sense with the theory of 
relativity, which 
is contained in 17,;‘ct (not able to produce nothing 
except what is present an opposition (aptil) and 
hand were valid arguments whenever two parts of our own 
body was visible: we ceinst criticism 
themselves, the relations between the necessary coiscient is 
practical, and it will be differences or “ substance,” a
clocks: 

not before ve a certain 
memory-image of the difference between the aspects of o